In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
import os
import math

low_memory=False
%config Completer.use_jedi = False

DATA_LOCATION = "data/combined/"

In [2]:
play_by_play = pd.DataFrame()
for filename in os.listdir(DATA_LOCATION):
    if(filename == "desktop.ini"):
        continue
    current_play_by_play = pd.read_csv(DATA_LOCATION + filename, encoding = "ISO-8859-1")
    play_by_play = pd.concat([play_by_play, current_play_by_play])

C:\Users\justin\anaconda3\envs\shap\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\justin\anaconda3\envs\shap\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,38,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\justin\anaconda3\envs\shap\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\justin\anaconda3\envs\shap\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.use

In [3]:
play_by_play_teams = play_by_play.groupby(['game_id', 'team']).agg({ 'date':'count'}).reset_index() #this needs to be split and put back in so we have both teams
play_by_play_teams = play_by_play_teams.drop('date', 1)
play_by_play_teams_odd = play_by_play_teams.iloc[::2, :]
play_by_play_teams_even = play_by_play_teams.iloc[1::2, :]
play_by_play_teams_odd = play_by_play_teams_odd.rename(columns={"team": "teamA"}, errors="raise")
play_by_play_teams_even = play_by_play_teams_even.rename(columns={"team": "teamB"}, errors="raise")
play_by_play_teams_joined = pd.merge(play_by_play_teams_even,play_by_play_teams_odd,on='game_id')
#play_by_play_teams_joined.to_csv("play_by_play_teams_joined.csv")

play_by_play = pd.merge(play_by_play,play_by_play_teams_joined,on='game_id')
play_by_play['opponent_team'] = np.where(play_by_play.team== play_by_play.teamA, play_by_play.teamB, play_by_play.teamA)

#play_by_play.type.value_counts().to_dict()
play_by_play.loc[(play_by_play.reason == 'shooting'),'reason'] = 's.foul' #and one
play_by_play['data_set'] = play_by_play.data_set.str.lower()
play_by_play['reason'] = play_by_play.reason.str.lower()
play_by_play['reason'] = play_by_play.reason.str.strip()
play_by_play['reason'] = play_by_play.reason.str.replace("\n", "", regex=False)
play_by_play['type'] = play_by_play.type.str.lower()
play_by_play['type'] = play_by_play.type.str.strip()
play_by_play['type'] = play_by_play.type.str.replace(" of ", "/", regex=False)
play_by_play['type'] = play_by_play.type.str.replace("\n", "", regex=False)

play_by_play.converted_x = pd.to_numeric(play_by_play['converted_x'],errors='coerce')
play_by_play.converted_y = pd.to_numeric(play_by_play['converted_y'],errors='coerce')
play_by_play.shot_distance = pd.to_numeric(play_by_play['shot_distance'],errors='coerce')

#transform the xy values to half court
play_by_play['x'] = play_by_play.converted_x
play_by_play['y'] = play_by_play.converted_y
play_by_play.loc[play_by_play.converted_y >= (94/2), "y"] = 94-play_by_play.converted_y[play_by_play.converted_y >= (94/2)]
play_by_play.loc[play_by_play.converted_y >= (94/2), "x"] = 50-play_by_play.converted_x[play_by_play.converted_y >= (94/2)]

pd.unique(play_by_play.data_set)
play_by_play['season'] = play_by_play.data_set.str[0:4] #change this to the trailing season
pd.unique(play_by_play.season)
play_by_play['playoffs'] = False
play_by_play.loc[play_by_play.data_set.str.contains('playoffs', na=False), 'playoffs'] = True
play_by_play.playoffs.describe()
play_by_play['game_id'] =  (play_by_play['game_id'].astype(str).str.replace('\W', '')) #clean the game id - without this we were losing teams

C:\Users\justin\anaconda3\envs\shap\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


array(['2018-2019 regular season', '2018-19 regular season',
       '2019 playoff', '2017-2018 regular season',
       '2017-18 regular season', '2018 playoff',
       '2019-2020 regular season', '2019-20 regular season',
       '2019-20 playoffs', '2016-2017 regular season', '2017 playoff',
       '2020-21 regular season', '2020-21 playoffs'], dtype=object)

array(['2018', '2019', '2017', '2016', '2020'], dtype=object)

count     2908910
unique          2
top         False
freq      2831149
Name: playoffs, dtype: object

C:\Users\justin\anaconda3\envs\shap\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: The default value of regex will change from True to False in a future version.


In [4]:
play_by_play.reset_index(drop=True, inplace=True)
play_by_play['seq_mask'] = 1
play_by_play.loc[(play_by_play.type == 'sub'),'seq_mask'] = 0
play_by_play.loc[(play_by_play.type == 'unknown'),'seq_mask'] = 0
play_by_play.loc[(play_by_play.type == 'uknown'),'seq_mask'] = 0
play_by_play.loc[(play_by_play.type == 'timeout: regular'),'seq_mask'] = 0
play_by_play.loc[(play_by_play.type == 'violation: delay/game violation'),'seq_mask'] = 0
play_by_play.loc[(play_by_play.type == 'violation:lane'),'seq_mask'] = 0
play_by_play.loc[(play_by_play.type == 'violation:defensive goaltending'),'seq_mask'] = 0
play_by_play.loc[(play_by_play.type == 'violation: delay/game'),'seq_mask'] = 0
play_by_play.loc[(play_by_play.type == 'team rebound'),'seq_mask'] = 0

play_by_play['seq'] = play_by_play.seq_mask.cumsum()
play_by_play.loc[(play_by_play.seq_mask == 0),'seq'] = np.nan
#play_by_play['seq'] = play_by_play.index
#next type - don't include subs
play_by_play_type = play_by_play[play_by_play.seq_mask == 1][['type', 'seq']]
play_by_play_type = play_by_play_type.rename(columns={"type": "next_type"})
play_by_play_type['prev_seq'] =  play_by_play_type.seq-1
play_by_play_type = play_by_play_type.drop(columns=['seq'])
play_by_play = pd.merge(play_by_play, play_by_play_type, left_on='seq', right_on='prev_seq', how="left")
play_by_play = play_by_play.drop(columns=['prev_seq'])

#prev type - don't include subs
play_by_play_type = play_by_play[play_by_play.seq_mask == 1][['type', 'seq']]
play_by_play_type = play_by_play_type.rename(columns={"type": "prev_type"})
play_by_play_type['next_seq'] =  play_by_play_type.seq+1
play_by_play_type = play_by_play_type.drop(columns=['seq'])
play_by_play = pd.merge(play_by_play, play_by_play_type, left_on='seq', right_on='next_seq', how="left")
play_by_play = play_by_play.drop(columns=['next_seq'])

In [5]:
play_by_play['shooting_foul_type'] = ""
play_by_play.loc[(play_by_play.reason == 's.foul') & (play_by_play.next_type == 'free throw 1/1'),'shooting_foul_type'] = 'made foul' #and one
play_by_play.loc[(play_by_play.reason == 's.foul') & (play_by_play.next_type != 'free throw 1/1'),'shooting_foul_type'] = 'missed foul' #missed
play_by_play.shooting_foul_type.value_counts().to_dict()

play_by_play['and_one_foul'] = False
play_by_play.loc[(play_by_play.next_type == 'free throw 1/1') & (play_by_play.reason != "l.b.foul") & (play_by_play.type != "personal take foul") & (play_by_play.reason != "personal take foul") & (play_by_play.type != "away.from.play.foul") & (play_by_play.reason != "away.from.play.foul") & (play_by_play.type != "p.foul") & (play_by_play.type != "free throw technical") & (play_by_play.type != "uknown") & (play_by_play.reason != "unknown") & (play_by_play.reason != "turnover") & (play_by_play.reason != "unknown") & (play_by_play.reason != "free throw technical"),'and_one_foul'] = True #don't include away from play fouls or loose ball fouls
play_by_play.and_one_foul.value_counts().to_dict()


#we need to include and_ones on the row that includes the xy coordinates so that I can bin the and one foul shots
play_by_play['and_one'] = False
play_by_play.loc[(play_by_play.next_type == 's.foul') & (play_by_play.result == "made") & (play_by_play.shot_distance >= 0) ,'and_one'] = True #don't include away from play fouls or loose ball fouls
play_by_play.and_one.value_counts().to_dict()
#the above seems too small - need to check the shot_distance

#the following peforms the same for
#play_by_play['shooting_foul_made_miss'] = ""
#play_by_play.loc[(play_by_play.next_type == 's.foul') & (play_by_play.result == "made") ,'shooting_foul_made_miss'] = 'made foul' #and one
#play_by_play.loc[((play_by_play.next_type == 'free throw 1/2') | (play_by_play.next_type == 'free throw 1/3')) & (play_by_play.reason == "s.foul") ,'shooting_foul_made_miss'] = "missed foul" #don't include away from play fouls or loose ball fouls
#play_by_play.shooting_foul_made_miss.value_counts().to_dict()


{'': 2790227, 'missed foul': 90700, 'made foul': 27983}

{False: 2880226, True: 28684}

{False: 2871894, True: 37016}

In [6]:
#I believe the above code chunk is not needed as this one is more direct


#id s.fouls
play_by_play['s_foul'] = np.nan
play_by_play.loc[((play_by_play.type == "s.foul") | (play_by_play.reason == "s.foul")) ,'s_foul'] = 1
play_by_play['s_foul_id'] = play_by_play.s_foul.cumsum()
play_by_play['s_foul_id_plays'] = play_by_play.s_foul.replace(np.nan, 0).cumsum()

#attach free throws to s.fouls
play_by_play['number_foul_throws'] = np.nan
play_by_play.loc[(play_by_play.type == 'free throw 1/1')  ,'number_foul_throws'] = 1
play_by_play.loc[(play_by_play.type == 'free throw 1/2')  ,'number_foul_throws'] = 2
play_by_play.loc[(play_by_play.type == 'free throw 1/3')  ,'number_foul_throws'] = 3
shooting_foul_free_throws = play_by_play.groupby('s_foul_id_plays').agg({ 'number_foul_throws':'first'}).reset_index()
shooting_foul_free_throws = shooting_foul_free_throws.rename(columns={"s_foul_id_plays": "s_foul_id"})
shooting_foul_free_throws.number_foul_throws.value_counts().to_dict()
play_by_play = play_by_play.drop(columns=['number_foul_throws'])
play_by_play = pd.merge(play_by_play, shooting_foul_free_throws, left_on='s_foul_id', right_on='s_foul_id', how="left")


#id shots including missed fouled shots (these are not listed as missed in the pbp data)
play_by_play['shot_result'] = np.nan
play_by_play.loc[(play_by_play.result == "made") & (play_by_play.shot_distance >= 0) ,'shot_result'] = "made"
play_by_play.loc[(play_by_play.result == "missed") & (play_by_play.shot_distance >= 0) ,'shot_result'] = "missed"
play_by_play.loc[(play_by_play.number_foul_throws > 1)  ,'shot_result'] = "missed_and_fouled"
play_by_play['shot_play'] = np.nan
play_by_play.loc[(((play_by_play.shot_result == "made") | (play_by_play.shot_result == "missed")) & (play_by_play.shot_distance >= 0)) |  (play_by_play.shot_result == "missed_and_fouled") ,'shot_play'] = 1
play_by_play['fga'] = np.nan
play_by_play.loc[(((play_by_play.shot_result == "made") | (play_by_play.shot_result == "missed")) & (play_by_play.shot_distance >= 0))  ,'fga'] = 1

play_by_play['shot_play_id'] = play_by_play.shot_play.cumsum()
play_by_play['shot_play_id_plays'] = play_by_play.shot_play.replace(np.nan, 0).cumsum()


#connect s.fouls to shots
shots = play_by_play.groupby('shot_play_id_plays').agg({ 'number_foul_throws':'first'}).reset_index()
shots = shots.rename(columns={"shot_play_id_plays": "shot_play_id"})
shots.number_foul_throws.value_counts().to_dict()
play_by_play = play_by_play.drop(columns=['number_foul_throws'])
play_by_play = pd.merge(play_by_play, shots, left_on='shot_play_id', right_on='shot_play_id', how="left")
play_by_play.number_foul_throws.value_counts().to_dict()

#create the columns necessary for binning
play_by_play['player_shooting'] = np.where(play_by_play.shot_result== 'missed_and_fouled', play_by_play.opponent, np.nan)
play_by_play['player_shooting'] = np.where((play_by_play.shot_result== 'missed') | (play_by_play.shot_result== 'made'), play_by_play.player, play_by_play.player_shooting)
#play_by_play = play_by_play.drop(columns=['player_shooting_missed_fouled'])

#create dummys for binning
play_by_play = play_by_play.reset_index(drop=True)
play_by_play = play_by_play.join(pd.get_dummies(play_by_play['shot_result'], prefix="shot_result"))

play_by_play['and_one_shot'] = 0
play_by_play.loc[(play_by_play.shot_result == "made") & (play_by_play.number_foul_throws == 1) ,'and_one_shot'] = 1

play_by_play['x_court'] = (play_by_play['x'] - 25) * 10 #translate so 0,0 is center of goal #https://www.bigdataball.com/datasets/nba/play-by-play/
play_by_play['y_court'] = (play_by_play['y'] - 5.25) * 10

{2.0: 85259, 1.0: 28313, 3.0: 5029}

{2.0: 85259, 1.0: 28307, 3.0: 5029}

{2.0: 85259, 1.0: 28307, 3.0: 5029}

In [7]:
#estimated total points from bin  = total points on fgm from bin + estimated points from ft made on “and ones” after made fg from bin + estimated points from ft made after missed fouled fg
#estimated points from ft made on “and ones” after made fg from bin =  Bin “and ones” shots * player ft%
#estimated points from ft made after missed fouled fg = Bin “and ones” shots * (Total fouled missed shots / Total “and ones” shots) * ft attempts given if fouled from bin on a missed shots * player ft%


#estimated total shots from bin  = fga from bin + estimated missed fouled shots
#estimated missed fouled shots  = Bin “and ones” shots * (Total fouled missed shots / Total “and ones” shots)





In [8]:
play_by_play.to_csv("cleanedData/play_by_play.csv")

In [9]:
#https://github.com/coryfoo/hexbins/blob/master/hexbin/binner.py
SQRT3 = 1.73205080756887729352744634150587236
scale = 20

def _nearest_bins_centerpoint(value: float, scale: float):
    """
    Returns the nearest two centerpoint positions (along the same axis) for the
    closest bins to the specified value.
    :param value: Either the x or y value of the point
    :param scale: The scale of the hexagons
    :returns: tuple of closest bin centerpoints
    """
    div, mod = divmod(value, scale / 2)
    rounded = scale / 2 * (div + (1 if div % 2 == 1 else 0))
    rounded_scaled = scale / 2 * (div + (1 if div % 2 == 0 else 0))
    return [rounded, rounded_scaled]


def _distance(x: float, y: float, x1: float, y1: float):
    """
    Returns the Euclidean distance from [x,y] to [x1,y1]
    """
    return math.sqrt((x - x1) * (x - x1) + (y - y1) * (y - y1))


def gen_bins(data, bin_width=1):
    """
    From the given array of data points ([[x, y], ...]), generate an array of
    hexagonal bins of given bin_width
    :param data: data point array
    :param bin_width: the size of the bins to create
    """
    bins = []
    bins_x = []
    bins_y = []

    for index, row in data.iterrows():
        x, y = row['x_court'], row['y_court']

        if pd.isnull(x):
            bin = np.nan
            bin_center_x = np.nan
            bin_center_y = np.nan
        else:

            px_nearest = _nearest_bins_centerpoint(x, bin_width)
            py_nearest = _nearest_bins_centerpoint(y, bin_width * SQRT3)

            z1 = _distance(x, y, px_nearest[0], py_nearest[0])
            z2 = _distance(x, y, px_nearest[1], py_nearest[1])

            if z1 < z2:
                bin = str([px_nearest[0], py_nearest[0]])
                bin_center_x = px_nearest[0]
                bin_center_y = py_nearest[0]
            else:
                bin = str([px_nearest[1], py_nearest[1]])
                bin_center_x = px_nearest[1]
                bin_center_y = py_nearest[1]

        bins.append(bin)
        bins_x.append(bin_center_x)
        bins_y.append(bin_center_y)

    return bins,bins_x,bins_y





play_by_play_bins, play_by_play_bins_center_x, play_by_play_bins_center_y = gen_bins(play_by_play, scale)
play_by_play['bin'] = play_by_play_bins
play_by_play['x_court_bin'] = play_by_play_bins_center_x
play_by_play['y_court_bin'] = play_by_play_bins_center_y

In [10]:
play_by_play_free_throws = play_by_play[play_by_play.event_type=="free throw"].copy()
play_by_play_free_throws['ftm'] = 0
play_by_play_free_throws.loc[(play_by_play_free_throws.result == 'made'),'ftm'] = 1
play_by_play_free_throws['fta'] = 1
play_by_play_free_throws_summary = play_by_play_free_throws.groupby(['player', 'season', 'playoffs']).agg({ 'ftm':'sum', 'fta':'sum'}).reset_index()
play_by_play_free_throws_summary['ft_percent'] = play_by_play_free_throws_summary['ftm']  / play_by_play_free_throws_summary['fta']
play_by_play_free_throws_summary = play_by_play_free_throws_summary.rename(columns={"player": "player_shooting"})

play_by_play_shots = play_by_play[play_by_play.shot_play==1].copy()
play_by_play_shots['missed_and_fouled'] = 0
play_by_play_shots.loc[(play_by_play_shots.shot_result == 'missed_and_fouled'),'missed_and_fouled'] = 1
play_by_play_shots_summary = play_by_play_shots.groupby(['player_shooting',  'season', 'playoffs']).agg({ 'and_one_shot':'sum', 'missed_and_fouled':'sum'}).reset_index()
play_by_play_shots_summary = play_by_play_shots_summary.rename(columns={"and_one_shot": "total_and_one_shot"})



In [11]:


play_by_play.loc[(play_by_play.shot_play == 1),'fta_given_if_fouled_and_missed'] = 2
play_by_play.loc[(play_by_play.shot_play == 1) & (play_by_play['description'].str.contains("3PT")),'fta_given_if_fouled_and_missed'] = 3

#estimated points from ft made on “and ones” after made fg from bin =  Bin “and ones” shots * player ft%
play_by_play_bins = play_by_play.groupby(['x_court_bin','y_court_bin', 'player_shooting', 'season', 'playoffs']).agg({ 'points':'sum', 'shot_play':'count','fga':'count', 'and_one_shot':'sum', 'shot_result_missed_and_fouled':'sum', 'shot_result_missed':'sum', 'shot_result_made':'sum', "fta_given_if_fouled_and_missed":"mean", "shot_play":"sum"}).reset_index()
#play_by_play_bins = play_by_play_bins.rename(columns={"shot_play": "fga"})
play_by_play_bins['fga'] = play_by_play_bins['fga'].fillna(0)
play_by_play_bins['fg_percentage'] = play_by_play_bins['shot_result_made'] / play_by_play_bins['fga']


play_by_play_bins = pd.merge(play_by_play_bins, play_by_play_free_throws_summary, left_on=['player_shooting','season', 'playoffs'], right_on=['player_shooting','season', 'playoffs'], how="left")
play_by_play_bins['ftm'] = play_by_play_bins['ftm'].fillna(0)
play_by_play_bins['fta'] = play_by_play_bins['fta'].fillna(0)
play_by_play_bins['estimated_points_from_ft_from_and_ones'] = play_by_play_bins['and_one_shot'] * play_by_play_bins['ft_percent']
play_by_play_bins['estimated_points_from_ft_from_and_ones'] = play_by_play_bins['estimated_points_from_ft_from_and_ones'].fillna(0)


#estimated points from ft made after missed fouled fg = Bin “and ones” shots * (Total fouled missed shots / Total “and ones” shots) * ft attempts given if fouled from bin on a missed shots * player ft%
play_by_play_bins = pd.merge(play_by_play_bins, play_by_play_shots_summary, left_on=['player_shooting','season', 'playoffs'], right_on=['player_shooting','season', 'playoffs'], how="left")
play_by_play_bins['and_one_shot'] = play_by_play_bins['and_one_shot'].fillna(0)
play_by_play_bins['missed_and_fouled'] = play_by_play_bins['missed_and_fouled'].fillna(0)
play_by_play_bins['estimated_points_from_ft_made_after_missed_fouled'] = play_by_play_bins['and_one_shot'] * (play_by_play_bins['missed_and_fouled']/play_by_play_bins['total_and_one_shot']) * play_by_play_bins['fta_given_if_fouled_and_missed'] * play_by_play_bins['ft_percent']
play_by_play_bins['estimated_points_from_ft_made_after_missed_fouled'] = play_by_play_bins['estimated_points_from_ft_made_after_missed_fouled'].fillna(0)


#estimated missed fouled shots  = Bin “and ones” shots * (Total fouled missed shots / Total “and ones” shots)
play_by_play_bins['estimated_missed_fouled_shots'] = play_by_play_bins['and_one_shot']  * (play_by_play_bins['missed_and_fouled']/play_by_play_bins['total_and_one_shot'])
play_by_play_bins['estimated_missed_fouled_shots'] = play_by_play_bins['estimated_missed_fouled_shots'].fillna(0)

#estimated total points from bin  = total points on fgm from bin + estimated points from ft made on “and ones” after made fg from bin + estimated points from ft made after missed fouled fg
play_by_play_bins['estimated_points'] = play_by_play_bins['points'] + play_by_play_bins['estimated_points_from_ft_from_and_ones'] + play_by_play_bins['estimated_points_from_ft_made_after_missed_fouled']

#estimated shot attempts from bin  = fga from bin + estimated missed fouled shots
play_by_play_bins['estimated_shot_attempts'] = play_by_play_bins['fga'] + play_by_play_bins['estimated_missed_fouled_shots']

#similar to TS% per bin or eFG% per bin
play_by_play_bins['expected_points'] = play_by_play_bins['points'] / play_by_play_bins['fga']
play_by_play_bins['true_expected_points'] = play_by_play_bins['estimated_points'] / play_by_play_bins['estimated_shot_attempts']



play_by_play_bins.to_csv("cleanedData/play_by_play_bins.csv")

In [12]:
play_by_play_free_throws_summary_team_test = play_by_play_free_throws.groupby(['player', 'team', 'season', 'playoffs']).agg({ 'ftm':'sum', 'fta':'sum'}).reset_index()
play_by_play_free_throws_summary_test = play_by_play_free_throws.groupby(['player', 'season', 'playoffs']).agg({ 'ftm':'sum', 'fta':'sum'}).reset_index()

#below is the problem - there are fewer for the team than the non-team. So there are shots made without a team column. player_shooting is being fouled by the player who is on the other team

play_by_play_shots['player_shooting_team'] = play_by_play_shots['team']
play_by_play_shots['player_shooting_team'] = np.where((play_by_play_shots.missed_and_fouled== 1) , play_by_play_shots.opponent_team, play_by_play_shots.team) #create other team
play_by_play_shots_summary_team_test = play_by_play_shots.groupby(['player_shooting', 'player_shooting_team', 'season', 'playoffs']).agg({ 'and_one_shot':'sum', 'missed_and_fouled':'sum'}).reset_index()
play_by_play_shots.sort_values(by=['player_shooting'])[0:10000].to_csv("play_by_play_shots_example.csv")
play_by_play_shots_summary_test = play_by_play_shots.groupby(['player_shooting',  'season', 'playoffs']).agg({ 'and_one_shot':'sum', 'missed_and_fouled':'sum'}).reset_index()



play_by_play_bins_team_test = play_by_play.groupby(['x_court_bin','y_court_bin', 'player_shooting', 'team', 'season', 'playoffs']).agg({ 'points':'sum', 'shot_play':'count', 'and_one_shot':'sum', 'shot_result_missed_and_fouled':'sum', 'shot_result_missed':'sum', 'shot_result_made':'sum', "fta_given_if_fouled_and_missed":"mean", "shot_play":"sum"}).reset_index()
play_by_play_bins_test = play_by_play.groupby(['x_court_bin','y_court_bin', 'player_shooting', 'season', 'playoffs']).agg({ 'points':'sum', 'shot_play':'count', 'and_one_shot':'sum', 'shot_result_missed_and_fouled':'sum', 'shot_result_missed':'sum', 'shot_result_made':'sum', "fta_given_if_fouled_and_missed":"mean", "shot_play":"sum"}).reset_index()


In [13]:
#find shot chart data for entire league
# Define a lambda function to compute the weighted mean:
#wm = lambda x: np.average(x, weights=play_by_play_bins.loc[x.index, "fga"])
wm = lambda x: np.average(x[~np.isnan(x)], weights=play_by_play_bins.loc[x[~np.isnan(x)].index, "fga"])
#wm = lambda x: np.average(x[~np.isnan(x)], weights=play_by_play_bins.loc[~np.isnan(x), "fga"])
#play_by_play_season_bins = play_by_play_bins.groupby(['x_court_bin','y_court_bin', 'season', 'playoffs']).agg(expected_points=('expected_points', wm), true_expected_points=('true_expected_points', wm), estimated_shot_attempts=('estimated_shot_attempts', wm),fga=('fga', wm), ).reset_index()
#play_by_play_season_bins.to_csv("cleanedData/play_by_play_season_bins.csv")

play_by_play_season_bins = play_by_play_bins.groupby(['x_court_bin','y_court_bin', 'season', 'playoffs']).agg(points=('points', sum), fga=('fga', sum), estimated_points=('estimated_points', sum),estimated_shot_attempts=('estimated_shot_attempts', sum), ).reset_index()
play_by_play_season_bins['true_expected_points'] = play_by_play_season_bins['estimated_points'] / play_by_play_season_bins['estimated_shot_attempts']
play_by_play_season_bins['expected_points'] = play_by_play_season_bins['points'] / play_by_play_season_bins['fga']
play_by_play_season_bins.to_csv("cleanedData/play_by_play_season_bins.csv")


In [14]:
play_by_play['missed_and_fouled'] = 0
play_by_play.loc[(play_by_play.shot_result == 'missed_and_fouled'),'missed_and_fouled'] = 1
play_by_play['player_shooting_team'] = play_by_play['team']
play_by_play['player_shooting_team'] = np.where((play_by_play.missed_and_fouled== 1) , play_by_play.opponent_team, play_by_play.team) #create other team

play_by_play_free_throws = play_by_play[play_by_play.event_type=="free throw"].copy()
play_by_play_free_throws['ftm'] = 0
play_by_play_free_throws.loc[(play_by_play_free_throws.result == 'made'),'ftm'] = 1
play_by_play_free_throws['fta'] = 1
play_by_play_free_throws['team'] = ""
play_by_play_free_throws['team'] = play_by_play_free_throws['player_shooting_team']
play_by_play_free_throws_summary = play_by_play_free_throws.groupby(['player', 'team', 'season', 'playoffs']).agg({ 'ftm':'sum', 'fta':'sum'}).reset_index()
play_by_play_free_throws_summary['ft_percent'] = play_by_play_free_throws_summary['ftm']  / play_by_play_free_throws_summary['fta']
play_by_play_free_throws_summary = play_by_play_free_throws_summary.rename(columns={"player": "player_shooting"})

play_by_play_shots = play_by_play[play_by_play.shot_play==1].copy()

#current_player = "James Harden"
#current_season = "2019"
#current_team = "HOU"
#play_by_play_shots_harden = play_by_play_shots[(play_by_play_shots.player_shooting == current_player) & (play_by_play_shots.season == current_season)& (play_by_play_shots.player_shooting_team == current_team)&(play_by_play_shots.playoffs == False)]
#play_by_play_shots_harden.to_csv("play_by_play_team_shots.csv")

#play_by_play_shots_harden = play_by_play_shots[(play_by_play_shots.player_shooting == current_player) & (play_by_play_shots.season == current_season)&(play_by_play_shots.playoffs == False)]
#play_by_play_shots_harden.to_csv("play_by_play_shots.csv")

play_by_play_shots_summary_team = play_by_play_shots.groupby(['player_shooting', 'player_shooting_team', 'season', 'playoffs']).agg({ 'and_one_shot':'sum', 'missed_and_fouled':'sum'}).reset_index()
play_by_play_shots_summary_team = play_by_play_shots_summary_team.rename(columns={"and_one_shot": "total_and_one_shot", "player_shooting_team":"team"})
#problem is for harden it's 284 vs 183

#play_by_play_shots_summary = play_by_play_shots.groupby(['player_shooting', 'team', 'season', 'playoffs']).agg({ 'and_one_shot':'sum', 'missed_and_fouled':'sum'}).reset_index()
#play_by_play_shots_summary = play_by_play_shots_summary.rename(columns={"and_one_shot": "total_and_one_shot", "player_shooting":"player"})

play_by_play_bins = play_by_play.groupby(['x_court_bin','y_court_bin', 'player_shooting', 'team', 'season', 'playoffs']).agg({ 'points':'sum', 'fga':'count', 'shot_play':'count', 'and_one_shot':'sum', 'shot_result_missed_and_fouled':'sum', 'shot_result_missed':'sum', 'shot_result_made':'sum', "fta_given_if_fouled_and_missed":"mean", "shot_play":"sum"}).reset_index()
#play_by_play_bins = play_by_play_bins.rename(columns={"shot_play": "fga"})
play_by_play_bins['fga'] = play_by_play_bins['fga'].fillna(0)
play_by_play_bins['fg_percentage'] = play_by_play_bins['shot_result_made'] / play_by_play_bins['fga']


play_by_play_bins = pd.merge(play_by_play_bins, play_by_play_free_throws_summary, left_on=['player_shooting','team','season',  'playoffs'], right_on=['player_shooting','team', 'season', 'playoffs'], how="left")
play_by_play_bins['ftm'] = play_by_play_bins['ftm'].fillna(0)
play_by_play_bins['fta'] = play_by_play_bins['fta'].fillna(0)
play_by_play_bins['estimated_points_from_ft_from_and_ones'] = play_by_play_bins['and_one_shot'] * play_by_play_bins['ft_percent']
play_by_play_bins['estimated_points_from_ft_from_and_ones'] = play_by_play_bins['estimated_points_from_ft_from_and_ones'].fillna(0)


#estimated points from ft made after missed fouled fg = Bin “and ones” shots * (Total fouled missed shots / Total “and ones” shots) * ft attempts given if fouled from bin on a missed shots * player ft%
play_by_play_bins = pd.merge(play_by_play_bins, play_by_play_shots_summary_team, left_on=['player_shooting','team', 'season', 'playoffs'], right_on=['player_shooting','team', 'season', 'playoffs'], how="left")
play_by_play_bins['and_one_shot'] = play_by_play_bins['and_one_shot'].fillna(0)
play_by_play_bins['missed_and_fouled'] = play_by_play_bins['missed_and_fouled'].fillna(0)
play_by_play_bins['estimated_points_from_ft_made_after_missed_fouled'] = play_by_play_bins['and_one_shot'] * (play_by_play_bins['missed_and_fouled']/play_by_play_bins['total_and_one_shot']) * play_by_play_bins['fta_given_if_fouled_and_missed'] * play_by_play_bins['ft_percent']
play_by_play_bins['estimated_points_from_ft_made_after_missed_fouled'] = play_by_play_bins['estimated_points_from_ft_made_after_missed_fouled'].fillna(0)


#estimated missed fouled shots  = Bin “and ones” shots * (Total fouled missed shots / Total “and ones” shots)
play_by_play_bins['estimated_missed_fouled_shots'] = play_by_play_bins['and_one_shot']  * (play_by_play_bins['missed_and_fouled']/play_by_play_bins['total_and_one_shot'])
play_by_play_bins['estimated_missed_fouled_shots'] = play_by_play_bins['estimated_missed_fouled_shots'].fillna(0)

#estimated total points from bin  = total points on fgm from bin + estimated points from ft made on “and ones” after made fg from bin + estimated points from ft made after missed fouled fg
play_by_play_bins['estimated_points'] = play_by_play_bins['points'] + play_by_play_bins['estimated_points_from_ft_from_and_ones'] + play_by_play_bins['estimated_points_from_ft_made_after_missed_fouled']

#estimated shot attempts from bin  = fga from bin + estimated missed fouled shots
play_by_play_bins['estimated_shot_attempts'] = play_by_play_bins['fga'] + play_by_play_bins['estimated_missed_fouled_shots']

#similar to TS% per bin or eFG% per bin
play_by_play_bins['expected_points'] = play_by_play_bins['points'] / play_by_play_bins['fga']
play_by_play_bins['true_expected_points'] = play_by_play_bins['estimated_points'] / play_by_play_bins['estimated_shot_attempts']
#find shot chart data for entire league
# Define a lambda function to compute the weighted mean:

#wm = lambda x: np.average(x[~np.isnan(x)], weights=play_by_play_bins.loc[~np.isnan(x), "fga"])
wm = lambda x: np.average(x[~np.isnan(x)], weights=play_by_play_bins.loc[x[~np.isnan(x)].index, "fga"])
#wm = lambda x: np.average(x, weights=play_by_play_bins.loc[x.index, "fga"])

#play_by_play_season_bins = play_by_play_bins.groupby(['x_court_bin','y_court_bin', 'team', 'season', 'playoffs']).agg(expected_points=('expected_points', wm), true_expected_points=('true_expected_points', wm), estimated_shot_attempts=('estimated_shot_attempts', wm),fga=('fga', wm), ).reset_index()
play_by_play_bins.to_csv("cleanedData/play_by_play_season_player_team_bins.csv")

play_by_play_season_team_bins = play_by_play_bins.groupby(['x_court_bin','y_court_bin', 'team', 'season', 'playoffs']).agg(points=('points', sum), fga=('fga', sum), estimated_points=('estimated_points', sum),estimated_shot_attempts=('estimated_shot_attempts', sum), ).reset_index()
play_by_play_season_team_bins['true_expected_points'] = play_by_play_season_team_bins['estimated_points'] / play_by_play_season_team_bins['estimated_shot_attempts']
play_by_play_season_team_bins['expected_points'] = play_by_play_season_team_bins['points'] / play_by_play_season_team_bins['fga']
play_by_play_season_team_bins.to_csv("cleanedData/play_by_play_season_team_bins.csv")



In [15]:
play_by_play_copy = play_by_play[play_by_play.game_id != "nan"].copy()

play_by_play_free_throws_summary = play_by_play_free_throws.groupby(['player', 'team', 'season', 'playoffs']).agg({ 'ftm':'sum', 'fta':'sum'}).reset_index()
play_by_play_free_throws_summary['ft_percent'] = play_by_play_free_throws_summary['ftm']  / play_by_play_free_throws_summary['fta']
play_by_play_free_throws_summary = play_by_play_free_throws_summary.rename(columns={"player": "player_shooting"})

play_by_play_shots = play_by_play_copy[play_by_play.shot_play==1].copy()

#current_player = "James Harden"
#current_season = "2019"
#current_team = "HOU"
#play_by_play_shots_harden = play_by_play_shots[(play_by_play_shots.player_shooting == current_player) & (play_by_play_shots.season == current_season)& (play_by_play_shots.player_shooting_team == current_team)&(play_by_play_shots.playoffs == False)]
#play_by_play_shots_harden.to_csv("play_by_play_team_shots.csv")

#play_by_play_shots_harden = play_by_play_shots[(play_by_play_shots.player_shooting == current_player) & (play_by_play_shots.season == current_season)&(play_by_play_shots.playoffs == False)]
#play_by_play_shots_harden.to_csv("play_by_play_shots.csv")

play_by_play_shots_summary_team = play_by_play_shots.groupby(['player_shooting', 'player_shooting_team', 'season', 'playoffs']).agg({ 'and_one_shot':'sum', 'missed_and_fouled':'sum'}).reset_index()
play_by_play_shots_summary_team = play_by_play_shots_summary_team.rename(columns={"and_one_shot": "total_and_one_shot", "player_shooting_team":"team"})
#problem is for harden it's 284 vs 183

#play_by_play_shots_summary = play_by_play_shots.groupby(['player_shooting', 'team', 'season', 'playoffs']).agg({ 'and_one_shot':'sum', 'missed_and_fouled':'sum'}).reset_index()
#play_by_play_shots_summary = play_by_play_shots_summary.rename(columns={"and_one_shot": "total_and_one_shot", "player_shooting":"player"})

play_by_play_bins = play_by_play_copy.groupby(['x_court_bin','y_court_bin', 'player_shooting','game_id',  'team', 'opponent_team', 'season', 'playoffs']).agg({ 'points':'sum','fga':'sum', 'shot_play':'count', 'and_one_shot':'sum', 'shot_result_missed_and_fouled':'sum', 'shot_result_missed':'sum', 'shot_result_made':'sum', "fta_given_if_fouled_and_missed":"mean", "shot_play":"sum"}).reset_index()
#play_by_play_bins = play_by_play_bins.rename(columns={"shot_play": "fga"})
play_by_play_bins['fga'] = play_by_play_bins['fga'].fillna(0)
play_by_play_bins['fg_percentage'] = play_by_play_bins['shot_result_made'] / play_by_play_bins['fga']


play_by_play_bins = pd.merge(play_by_play_bins, play_by_play_free_throws_summary, left_on=['player_shooting','team','season',  'playoffs'], right_on=['player_shooting','team', 'season', 'playoffs'], how="left")
play_by_play_bins['ftm'] = play_by_play_bins['ftm'].fillna(0)
play_by_play_bins['fta'] = play_by_play_bins['fta'].fillna(0)
play_by_play_bins['estimated_points_from_ft_from_and_ones'] = play_by_play_bins['and_one_shot'] * play_by_play_bins['ft_percent']
play_by_play_bins['estimated_points_from_ft_from_and_ones'] = play_by_play_bins['estimated_points_from_ft_from_and_ones'].fillna(0)


#estimated points from ft made after missed fouled fg = Bin “and ones” shots * (Total fouled missed shots / Total “and ones” shots) * ft attempts given if fouled from bin on a missed shots * player ft%
play_by_play_bins = pd.merge(play_by_play_bins, play_by_play_shots_summary_team, left_on=['player_shooting','team', 'season', 'playoffs'], right_on=['player_shooting','team', 'season', 'playoffs'], how="left")
play_by_play_bins['and_one_shot'] = play_by_play_bins['and_one_shot'].fillna(0)
play_by_play_bins['missed_and_fouled'] = play_by_play_bins['missed_and_fouled'].fillna(0)
play_by_play_bins['estimated_points_from_ft_made_after_missed_fouled'] = play_by_play_bins['and_one_shot'] * (play_by_play_bins['missed_and_fouled']/play_by_play_bins['total_and_one_shot']) * play_by_play_bins['fta_given_if_fouled_and_missed'] * play_by_play_bins['ft_percent']
play_by_play_bins['estimated_points_from_ft_made_after_missed_fouled'] = play_by_play_bins['estimated_points_from_ft_made_after_missed_fouled'].fillna(0)


#estimated missed fouled shots  = Bin “and ones” shots * (Total fouled missed shots / Total “and ones” shots)
play_by_play_bins['estimated_missed_fouled_shots'] = play_by_play_bins['and_one_shot']  * (play_by_play_bins['missed_and_fouled']/play_by_play_bins['total_and_one_shot'])
play_by_play_bins['estimated_missed_fouled_shots'] = play_by_play_bins['estimated_missed_fouled_shots'].fillna(0)

#estimated total points from bin  = total points on fgm from bin + estimated points from ft made on “and ones” after made fg from bin + estimated points from ft made after missed fouled fg
play_by_play_bins['estimated_points'] = play_by_play_bins['points'] + play_by_play_bins['estimated_points_from_ft_from_and_ones'] + play_by_play_bins['estimated_points_from_ft_made_after_missed_fouled']

#estimated shot attempts from bin  = fga from bin + estimated missed fouled shots
play_by_play_bins['estimated_shot_attempts'] = play_by_play_bins['fga'] + play_by_play_bins['estimated_missed_fouled_shots']

#similar to TS% per bin or eFG% per bin
play_by_play_bins['expected_points'] = play_by_play_bins['points'] / play_by_play_bins['fga']
play_by_play_bins['true_expected_points'] = play_by_play_bins['estimated_points'] / play_by_play_bins['estimated_shot_attempts']


#we need to include ftm within points
play_by_play_season_game_team_bins = play_by_play_bins.groupby(['x_court_bin','y_court_bin', 'game_id', 'team', 'opponent_team', 'season', 'playoffs']).agg(points=('points', sum), fga=('fga', sum), estimated_points=('estimated_points', sum),estimated_shot_attempts=('estimated_shot_attempts', sum), ).reset_index()
play_by_play_season_game_team_bins['true_expected_points'] = play_by_play_season_game_team_bins['estimated_points'] / play_by_play_season_game_team_bins['estimated_shot_attempts']
play_by_play_season_game_team_bins['expected_points'] = play_by_play_season_game_team_bins['points'] / play_by_play_season_game_team_bins['fga']

play_by_play_season_game_team_bins_opponent = play_by_play_season_game_team_bins.copy()
play_by_play_season_game_team_bins_opponent = play_by_play_season_game_team_bins_opponent.rename(columns={'team':'opponent_team','opponent_team':'team','estimated_shot_attempts':'opponent_estimated_shot_attempts','estimated_points':'opponent_estimated_points','fga':'opponent_fga','points':'opponent_points','true_expected_points':'opponent_true_expected_points','expected_points':'opponent_expected_points'})
play_by_play_season_game_team_bins = pd.merge(play_by_play_season_game_team_bins,play_by_play_season_game_team_bins_opponent, on=['x_court_bin','y_court_bin','game_id', 'team', 'opponent_team', 'season', 'playoffs'],how='outer')




play_by_play_season_game_team_summary = play_by_play.groupby([ 'game_id', 'team', 'opponent_team', 'season', 'playoffs']).agg(points=('points', sum)).reset_index()
play_by_play_season_game_team_summary = play_by_play_season_game_team_summary[play_by_play_season_game_team_summary.game_id != "nan"]
play_by_play_season_game_team_summary_opponent = play_by_play_season_game_team_summary.copy()
play_by_play_season_game_team_summary_opponent = play_by_play_season_game_team_summary_opponent.rename(columns={'team':'opponent_team','opponent_team':'team','points':'opponent_points_game_total'})
play_by_play_season_game_team_summary = pd.merge(play_by_play_season_game_team_summary,play_by_play_season_game_team_summary_opponent, on=['game_id', 'team', 'opponent_team', 'season', 'playoffs'],how='left')
play_by_play_season_game_team_summary = play_by_play_season_game_team_summary.rename(columns={"points":"points_game_total"})
play_by_play_season_game_team_summary.to_csv("play_by_play_season_game_team_summary.csv")
play_by_play_season_game_team_bins = pd.merge(play_by_play_season_game_team_bins, play_by_play_season_game_team_summary, left_on=['game_id','team','opponent_team','season','playoffs'], right_on=['game_id','team','opponent_team','season','playoffs'], how="left")



play_by_play_season_game_team_bin_summary = play_by_play_bins.groupby([ 'game_id', 'team', 'opponent_team', 'season', 'playoffs']).agg(estimated_shot_attempts=('estimated_shot_attempts', sum), fga=('fga', sum) ).reset_index()
play_by_play_season_game_team_bin_summary = play_by_play_season_game_team_bin_summary[play_by_play_season_game_team_bin_summary.game_id != "nan"]
play_by_play_season_game_team_bin_summary_opponent = play_by_play_season_game_team_bin_summary.copy()
play_by_play_season_game_team_bin_summary_opponent = play_by_play_season_game_team_bin_summary_opponent.rename(columns={'team':'opponent_team','opponent_team':'team','estimated_shot_attempts':'opponent_estimated_shot_attempts_game_total','fga':'opponent_fga_game_total',})
play_by_play_season_game_team_bin_summary = pd.merge(play_by_play_season_game_team_bin_summary,play_by_play_season_game_team_bin_summary_opponent, on=['game_id', 'team', 'opponent_team', 'season', 'playoffs'])
play_by_play_season_game_team_bin_summary = play_by_play_season_game_team_bin_summary.rename(columns={"estimated_shot_attempts":"estimated_shot_attempts_game_total", "fga":"fga_game_total"})
play_by_play_season_game_team_bin_summary.to_csv("play_by_play_season_game_team_bin_summary.csv")
play_by_play_season_game_team_bins = pd.merge(play_by_play_season_game_team_bins, play_by_play_season_game_team_bin_summary, left_on=['game_id','team','opponent_team', 'season','playoffs'], right_on=['game_id','team','opponent_team','season','playoffs'], how="left")



play_by_play_season_game_team_bins.to_csv("cleanedData/play_by_play_season_game_team_bins.csv")

In [16]:

play_by_play_season_game_team_bins_opponent = play_by_play_season_game_team_bins.copy()
play_by_play_season_game_team_bins_opponent = play_by_play_season_game_team_bins_opponent.rename(columns={'team':'opponent_team','opponent_team':'team','estimated_shot_attempts':'opponent_estimated_shot_attempts','estimated_points':'opponent_estimated_points','fga':'opponent_fga','points':'opponent_points','true_expected_points':'opponent_true_expected_points','expected_points':'opponent_expected_points'})
#play_by_play_season_game_team_bins = pd.merge(play_by_play_season_game_team_bins,play_by_play_season_game_team_bins_opponent, on=['game_id', 'team', 'opponent_team', 'season', 'playoffs'])

play_by_play_season_game_team_bins_opponent

,x_court_bin,y_court_bin,game_id,opponent_team,team,season,playoffs,opponent_points,opponent_fga,opponent_estimated_points,...,opponent_estimated_points,opponent_estimated_shot_attempts,opponent_true_expected_points,opponent_expected_points,points_game_total,opponent_points_game_total,estimated_shot_attempts_game_total,fga_game_total,opponent_estimated_shot_attempts_game_total,opponent_fga_game_total
0,-250.0,-17.320508,0021600012,SAC,PHX,2016,False,3.0,1.0,3.0,...,NaN,NaN,NaN,NaN,113.0,94.0,93.563471,86.0,79.423077,77.0
1,-250.0,-17.320508,0021600013,HOU,LAL,2016,False,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,114.0,120.0,97.612479,85.0,94.306548,88.0
2,-250.0,-17.320508,0021600019,BKN,IND,2016,False,0.0,2.0,0.0,...,3.0,2.0,1.5,1.5,103.0,94.0,89.500000,87.0,106.375880,98.0
3,-250.0,-17.320508,0021600019,IND,BKN,2016,False,3.0,2.0,3.0,...,0.0,2.0,0.0,0.0,94.0,103.0,106.375880,98.0,89.500000,87.0
4,-250.0,-17.320508,0021600036,PHX,GSW,2016,False,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,100.0,106.0,93.676471,90.0,94.844444,82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177475,260.0,0.000000,0041600404,GSW,CLE,2017,False,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,116.0,137.0,90.447436,85.0,93.125000,87.0
1177476,260.0,0.000000,21700471,DET,NYK,2017,False,NaN,NaN,NaN,...,3.0,1.0,3.0,3.0,104.0,101.0,90.631579,88.0,102.857143,97.0
1177477,260.0,34.641016,0021600833,LAL,SAC,2016,False,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,96.0,97.0,82.000000,82.0,75.037736,72.0
1177478,260.0,34.641016,0021600894,CHA,LAL,2016,False,NaN,NaN,NaN,...,3.0,1.0,3.0,3.0,109.0,104.0,89.971429,85.0,89.648459,82.0


In [17]:
play_by_play_player_team_free_throws = play_by_play[play_by_play.event_type=="free throw"].copy()
play_by_play_player_team_free_throws['ftm'] = 0
play_by_play_player_team_free_throws.loc[(play_by_play_player_team_free_throws.result == 'made'),'ftm'] = 1
play_by_play_player_team_free_throws['fta'] = 1
play_by_play_player_team_free_throws['team'] = play_by_play_player_team_free_throws['player_shooting_team']
play_by_play_player_team_free_throws_summary = play_by_play_player_team_free_throws.groupby(['player', 'season', 'playoffs', 'team']).agg({ 'ftm':'sum', 'fta':'sum'}).reset_index()
play_by_play_player_team_free_throws_summary['ft_percent'] = play_by_play_player_team_free_throws_summary['ftm']  / play_by_play_player_team_free_throws_summary['fta']
play_by_play_player_team_free_throws_summary = play_by_play_player_team_free_throws_summary.rename(columns={"player": "player_shooting"})

play_by_play_shots_summary_team = play_by_play[play_by_play.shot_play==1].copy()
play_by_play_shots_summary_team['missed_and_fouled'] = 0
play_by_play_shots_summary_team.loc[(play_by_play_shots_summary_team.shot_result == 'missed_and_fouled'),'missed_and_fouled'] = 1
play_by_play_shots_summary_team = play_by_play_shots.groupby(['player_shooting', 'player_shooting_team', 'season', 'playoffs']).agg({ 'and_one_shot':'sum', 'missed_and_fouled':'sum'}).reset_index()
play_by_play_shots_summary_team = play_by_play_shots_summary_team.rename(columns={"and_one_shot": "total_and_one_shot", "player_shooting_team":"team"})

#find shot chart data divided by 3PT and 2PT
play_by_play.loc[(play_by_play.shot_play == 1),'fg_type'] = 2
play_by_play.loc[(play_by_play.shot_play == 1) & (play_by_play['description'].str.contains("3PT")),'fg_type'] = 3

#estimated points from ft made on “and ones” after made fg from bin =  Bin “and ones” shots * player ft%
play_by_play_fg_type = play_by_play.groupby(['fg_type', 'player_shooting','team', 'season', 'playoffs']).agg({ 'points':'sum', 'shot_play':'count', 'fga':'count', 'and_one_shot':'sum', 'shot_result_missed_and_fouled':'sum', 'shot_result_missed':'sum', 'shot_result_made':'sum', "fta_given_if_fouled_and_missed":"mean", "shot_play":"sum"}).reset_index()
#play_by_play_fg_type = play_by_play_fg_type.rename(columns={"shot_play": "fga"})
play_by_play_fg_type['fga'] = play_by_play_fg_type['fga'].fillna(0)
play_by_play_fg_type['fg_percentage'] = play_by_play_fg_type['shot_result_made'] / play_by_play_fg_type['fga']


play_by_play_fg_type = pd.merge(play_by_play_fg_type, play_by_play_player_team_free_throws_summary, left_on=['player_shooting','team','season', 'playoffs'], right_on=['player_shooting','team','season', 'playoffs'], how="left")
play_by_play_fg_type['ftm'] = play_by_play_fg_type['ftm'].fillna(0)
play_by_play_fg_type['fta'] = play_by_play_fg_type['fta'].fillna(0)
play_by_play_fg_type['estimated_points_from_ft_from_and_ones'] = play_by_play_fg_type['and_one_shot'] * play_by_play_fg_type['ft_percent']
play_by_play_fg_type['estimated_points_from_ft_from_and_ones'] = play_by_play_fg_type['estimated_points_from_ft_from_and_ones'].fillna(0)


#estimated points from ft made after missed fouled fg = Bin “and ones” shots * (Total fouled missed shots / Total “and ones” shots) * ft attempts given if fouled from bin on a missed shots * player ft%
play_by_play_fg_type = pd.merge(play_by_play_fg_type, play_by_play_shots_summary_team, left_on=['player_shooting','team','season','playoffs'], right_on=['player_shooting','team','season', 'playoffs'], how="left")
play_by_play_fg_type['and_one_shot'] = play_by_play_fg_type['and_one_shot'].fillna(0)
play_by_play_fg_type['missed_and_fouled'] = play_by_play_fg_type['missed_and_fouled'].fillna(0)
play_by_play_fg_type['estimated_points_from_ft_made_after_missed_fouled'] = play_by_play_fg_type['and_one_shot'] * (play_by_play_fg_type['missed_and_fouled']/play_by_play_fg_type['total_and_one_shot']) * play_by_play_fg_type['fta_given_if_fouled_and_missed'] * play_by_play_fg_type['ft_percent']
play_by_play_fg_type['estimated_points_from_ft_made_after_missed_fouled'] = play_by_play_fg_type['estimated_points_from_ft_made_after_missed_fouled'].fillna(0)


#estimated missed fouled shots  = Bin “and ones” shots * (Total fouled missed shots / Total “and ones” shots)
play_by_play_fg_type['estimated_missed_fouled_shots'] = play_by_play_fg_type['and_one_shot']  * (play_by_play_fg_type['missed_and_fouled']/play_by_play_fg_type['total_and_one_shot'])
play_by_play_fg_type['estimated_missed_fouled_shots'] = play_by_play_fg_type['estimated_missed_fouled_shots'].fillna(0)

#estimated total points from bin  = total points on fgm from bin + estimated points from ft made on “and ones” after made fg from bin + estimated points from ft made after missed fouled fg
play_by_play_fg_type['estimated_points'] = play_by_play_fg_type['points'] + play_by_play_fg_type['estimated_points_from_ft_from_and_ones'] + play_by_play_fg_type['estimated_points_from_ft_made_after_missed_fouled']

#estimated shot attempts from bin  = fga from bin + estimated missed fouled shots
play_by_play_fg_type['estimated_shot_attempts'] = play_by_play_fg_type['fga'] + play_by_play_fg_type['estimated_missed_fouled_shots']

#similar to TS% per bin or eFG% per bin
play_by_play_fg_type['expected_points'] = play_by_play_fg_type['points'] / play_by_play_fg_type['fga'] #I think this is wrong bc points is average and fga is sum
play_by_play_fg_type['true_expected_points'] = play_by_play_fg_type['estimated_points'] / play_by_play_fg_type['estimated_shot_attempts']
play_by_play_fg_type['true_shooting_over_expected'] = play_by_play_fg_type['true_expected_points']-play_by_play_fg_type['expected_points']


play_by_play_fg_type.to_csv("cleanedData/play_by_play_fg_type.csv")

In [18]:
play_by_play.loc[(play_by_play.result == "made") & (play_by_play.shot_distance >= 0) ,'shot_result'] = "made"

play_by_play[['remaining_time_h','remaining_time_m','remaining_time_sec']] = play_by_play['remaining_time'].str.split(':',expand=True)
play_by_play['remaining_time_h'] = play_by_play.remaining_time_h.astype(float)
play_by_play['remaining_time_m'] = play_by_play.remaining_time_m.astype(float)
play_by_play['remaining_time_s'] = play_by_play.remaining_time_h.astype(float)
play_by_play['crunch_time'] = False
play_by_play.loc[(play_by_play.remaining_time_m <= 2) & (play_by_play.period >= 4) ,'crunch_time'] = True

play_by_play_player_team_free_throws = play_by_play[play_by_play.event_type=="free throw"].copy()
play_by_play_player_team_free_throws['ftm'] = 0
play_by_play_player_team_free_throws.loc[(play_by_play_player_team_free_throws.result == 'made'),'ftm'] = 1
play_by_play_player_team_free_throws['fta'] = 1
play_by_play_player_team_free_throws['team'] = play_by_play_player_team_free_throws['player_shooting_team']
play_by_play_player_team_free_throws_summary = play_by_play_player_team_free_throws.groupby(['player', 'season','team', 'playoffs',  'period','crunch_time']).agg({ 'ftm':'sum', 'fta':'sum'}).reset_index()
play_by_play_player_team_free_throws_summary['ft_percent'] = play_by_play_player_team_free_throws_summary['ftm']  / play_by_play_player_team_free_throws_summary['fta']
play_by_play_player_team_free_throws_summary = play_by_play_player_team_free_throws_summary.rename(columns={"player": "player_shooting"})

play_by_play_shots = play_by_play[play_by_play.shot_play==1].copy()
play_by_play_shots['missed_and_fouled'] = 0
play_by_play_shots.loc[(play_by_play_shots.shot_result == 'missed_and_fouled'),'missed_and_fouled'] = 1
play_by_play_shots_summary = play_by_play_shots.groupby(['player_shooting','player_shooting_team', 'season', 'period','crunch_time', 'playoffs']).agg({ 'and_one_shot':'sum', 'missed_and_fouled':'sum'}).reset_index()
play_by_play_shots_summary = play_by_play_shots_summary.rename(columns={"and_one_shot": "total_and_one_shot", "player_shooting_team":"team"})



#find shot chart data divided by 3PT and 2PT
play_by_play.loc[(play_by_play.shot_play == 1),'fg_type'] = 2
play_by_play.loc[(play_by_play.shot_play == 1) & (play_by_play['description'].str.contains("3PT")),'fg_type'] = 3

#estimated points from ft made on “and ones” after made fg from bin =  Bin “and ones” shots * player ft%
play_by_play_fg_type = play_by_play.groupby(['fg_type', 'player_shooting','team', 'season', 'playoffs','period','crunch_time']).agg({ 'points':'sum', 'fga':'count', 'shot_play':'count', 'and_one_shot':'sum', 'shot_result_missed_and_fouled':'sum', 'shot_result_missed':'sum', 'shot_result_made':'sum', "fta_given_if_fouled_and_missed":"mean", "shot_play":"sum"}).reset_index()
#play_by_play_fg_type = play_by_play_fg_type.rename(columns={"shot_play": "fga"})
play_by_play_fg_type['fga'] = play_by_play_fg_type['fga'].fillna(0)
play_by_play_fg_type['fg_percentage'] = play_by_play_fg_type['shot_result_made'] / play_by_play_fg_type['fga']


play_by_play_fg_type = pd.merge(play_by_play_fg_type, play_by_play_player_team_free_throws_summary, left_on=['player_shooting','team','season', 'period','crunch_time', 'playoffs'], right_on=['player_shooting','team','season', 'period','crunch_time', 'playoffs'], how="left")
play_by_play_fg_type['ftm'] = play_by_play_fg_type['ftm'].fillna(0)
play_by_play_fg_type['fta'] = play_by_play_fg_type['fta'].fillna(0)
play_by_play_fg_type['estimated_points_from_ft_from_and_ones'] = play_by_play_fg_type['and_one_shot'] * play_by_play_fg_type['ft_percent']
play_by_play_fg_type['estimated_points_from_ft_from_and_ones'] = play_by_play_fg_type['estimated_points_from_ft_from_and_ones'].fillna(0)


#estimated points from ft made after missed fouled fg = Bin “and ones” shots * (Total fouled missed shots / Total “and ones” shots) * ft attempts given if fouled from bin on a missed shots * player ft%
play_by_play_fg_type = pd.merge(play_by_play_fg_type, play_by_play_shots_summary, left_on=['player_shooting','team','season',  'period','crunch_time', 'playoffs'], right_on=['player_shooting','team','season', 'period','crunch_time','playoffs'], how="left")
play_by_play_fg_type['and_one_shot'] = play_by_play_fg_type['and_one_shot'].fillna(0)
play_by_play_fg_type['missed_and_fouled'] = play_by_play_fg_type['missed_and_fouled'].fillna(0)
play_by_play_fg_type['estimated_points_from_ft_made_after_missed_fouled'] = play_by_play_fg_type['and_one_shot'] * (play_by_play_fg_type['missed_and_fouled']/play_by_play_fg_type['total_and_one_shot']) * play_by_play_fg_type['fta_given_if_fouled_and_missed'] * play_by_play_fg_type['ft_percent']
play_by_play_fg_type['estimated_points_from_ft_made_after_missed_fouled'] = play_by_play_fg_type['estimated_points_from_ft_made_after_missed_fouled'].fillna(0)


#estimated missed fouled shots  = Bin “and ones” shots * (Total fouled missed shots / Total “and ones” shots)
play_by_play_fg_type['estimated_missed_fouled_shots'] = play_by_play_fg_type['and_one_shot']  * (play_by_play_fg_type['missed_and_fouled']/play_by_play_fg_type['total_and_one_shot'])
play_by_play_fg_type['estimated_missed_fouled_shots'] = play_by_play_fg_type['estimated_missed_fouled_shots'].fillna(0)

#estimated total points from bin  = total points on fgm from bin + estimated points from ft made on “and ones” after made fg from bin + estimated points from ft made after missed fouled fg
play_by_play_fg_type['estimated_points'] = play_by_play_fg_type['points'] + play_by_play_fg_type['estimated_points_from_ft_from_and_ones'] + play_by_play_fg_type['estimated_points_from_ft_made_after_missed_fouled']

#estimated shot attempts from bin  = fga from bin + estimated missed fouled shots
play_by_play_fg_type['estimated_shot_attempts'] = play_by_play_fg_type['fga'] + play_by_play_fg_type['estimated_missed_fouled_shots']

#similar to TS% per bin or eFG% per bin
play_by_play_fg_type['expected_points'] = play_by_play_fg_type['points'] / play_by_play_fg_type['fga'] #I think this is wrong bc points is average and fga is sum
play_by_play_fg_type['true_expected_points'] = play_by_play_fg_type['estimated_points'] / play_by_play_fg_type['estimated_shot_attempts']
play_by_play_fg_type['true_shooting_over_expected'] = play_by_play_fg_type['true_expected_points']-play_by_play_fg_type['expected_points']



play_by_play_fg_type.to_csv("cleanedData/play_by_play_fg_type_period.csv")

In [19]:
play_by_play_player_team_free_throws_summary.crunch_time

0        False
1        False
2         True
3        False
4        False
         ...  
14543    False
14544    False
14545     True
14546    False
14547     True
Name: crunch_time, Length: 14548, dtype: bool